In [1]:
!pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00


In [2]:
# import dependencies
from IPython.display import display, Javascript
from IPython.display import Image as ipython_img
from matplotlib import pyplot as plt
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch 
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
from PIL import Image
import tensorflow as tf
import pickle

# Import package

In [3]:
import cv2
from facenet_pytorch import MTCNN
import torch
import os

In [4]:
#check type of device (cpu, gpu, tpu)
device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Prepare data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/projects/ComputerVision/identifier_system_by_face'
%cd '{BASE_DIR}'
TRAIN_DATASET_DIR = os.path.join(BASE_DIR, 'train_dataset')
TEST_DATASET_DIR = os.path.join(BASE_DIR, 'test_dataset')
IMG_FOLDER = 'img'
FACE_FOLDER = 'faces'
EMBEDDING_FOLDER = 'embeddings'

Mounted at /content/drive
/content/drive/MyDrive/projects/ComputerVision/identifier_system_by_face


# For face detection

In [31]:
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)

In [ ]:
def extract_face(box, img, frame_size, margin=20):
    face_size = 160
    img_size = frame_size

    if box[0] > img_size[1] or box[2] > img_size[1]:
      print('Error face detection')
      return img, False

    margin = [
        margin * (box[2] - box[0]) / (face_size - margin),
        margin * (box[3] - box[1]) / (face_size - margin),
    ]
    margin_box = [ #box[0] và box[1] là tọa độ của điểm góc trên cùng trái
        int(max(box[0] - margin[0] / 2, 0)), #nếu thêm vào margin bị ra khỏi rìa ảnh => đưa về điểm 0
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, img_size[0])), #nếu thêm vào margin bị ra khỏi rìa ảnh => đưa về tọa độ của ảnh gốc
        int(min(box[3] + margin[1] / 2, img_size[1])),
    ] #tạo margin mới bao quanh box cũ
    try:
      margin_img = img[margin_box[1]:margin_box[3], margin_box[0]:margin_box[2], :]
      face = cv2.resize(margin_img,(face_size, face_size), interpolation=cv2.INTER_AREA)
    except:
      no_margin_img = img[box[1]:box[3], box[0]:box[2], :]
      face = cv2.resize(no_margin_img,(face_size, face_size), interpolation=cv2.INTER_AREA)


    #convert numpy array to image format
    #face = Img.fromarray(face)
    return face, True

def save_img_to_file(img, img_path):
  cv2.imwrite(img_path, img)

def transfrom_img(img):
  img = np.moveaxis(img, -1, 0)
  img = torch.from_numpy(img)
  return img/255

def extract_and_save_faces(dataset_dir):
  for folder in os.listdir(dataset_dir):
    img_dir = os.path.join(dataset_dir, folder, IMG_FOLDER)
    face_dir = os.path.join(dataset_dir, folder, FACE_FOLDER)
    if os.path.exists(face_dir) != True:
      os.mkdir(face_dir)
    for img_file in os.listdir(img_dir):
      img_path = os.path.join(img_dir, img_file)
      input_img = cv2.imread(img_path)
      boxes, _ = mtcnn.detect(input_img)
      if boxes is not None:
          for idx, box in enumerate(boxes):
              bbox = list(map(int,box.tolist()))
              face, status = extract_face(bbox, input_img, input_img.shape)
              if status == True:
                face_path = os.path.join(face_dir, img_file)
                try:
                  save_img_to_file(face, face_path)
                except:
                  img_file = f'{img_file.split(".")[0]}.jpg'
                  face_path = os.path.join(face_dir, img_file)
                  save_img_to_file(face, face_path)

# For pretrain model (pytorch) - Recognition

In [70]:
# Intialize embedding model
model = InceptionResnetV1(
    classify=False,
    pretrained="vggface2"
).to(device)

model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [71]:
def embedding_and_save(dataset_dir):
  # Embedding face and save it
  for folder in os.listdir(dataset_dir):
    face_dir = os.path.join(dataset_dir, folder, FACE_FOLDER)
    embedding_dir = os.path.join(dataset_dir, folder, EMBEDDING_FOLDER)
    if os.path.exists(embedding_dir) != True:
      os.mkdir(embedding_dir)
    embeds = []
    for face_file in os.listdir(face_dir):
        face_path = os.path.join(face_dir, face_file)
        try:
            img = cv2.imread(face_path)
        except:
            continue
        with torch.no_grad():
            img = transfrom_img(img).to(device)
            img = img.unsqueeze(0)
            embed = model(img)
            embeds.append(embed) #1 anh, kich thuoc [1,512]
        if len(embeds) == 0:
            continue
    embedding = torch.cat(embeds).mean(0, keepdim=True) #dua ra trung binh cua 50 anh, kich thuoc [1,512]
    embedding_path = os.path.join(embedding_dir, f'{folder}.pth')
    torch.save(embedding, embedding_path)

def prepare_data_for_training():
  print('PROCESSING TRAIN preparing data step')
  #training dataset
  extract_and_save_faces(TRAIN_DATASET_DIR)
  embedding_and_save(TRAIN_DATASET_DIR)
  print('DONE TRAIN preparing data step')

def prepare_data_for_testing():
  print('PROCESSING TEST preparing data step')
  #testing dataset
  extract_and_save_faces(TEST_DATASET_DIR)
  embedding_and_save(TEST_DATASET_DIR)
  print('DONE TEST preparing data step')


In [80]:
prepare_data_for_training()
prepare_data_for_testing()

PROCESSING TRAIN preparing data step
DONE TRAIN preparing data step
PROCESSING TEST preparing data step
DONE TEST preparing data step


# For my own model (tensorflow) - Recognition

**Load my own model**

In [42]:
!pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
MODEL_DIR = os.path.join(BASE_DIR, 'model')
model_name = 'face_recognition_triplot.h5'
model_path = os.path.join(MODEL_DIR, model_name)
model = tf.keras.models.load_model(model_path, compile=False)

In [45]:
def tf_embedding_and_save(dataset_dir):
  # Embedding face and save it
  for folder in os.listdir(dataset_dir):
    face_dir = os.path.join(dataset_dir, folder, FACE_FOLDER)
    embedding_dir = os.path.join(dataset_dir, folder, EMBEDDING_FOLDER)
    if os.path.exists(embedding_dir) != True:
      os.mkdir(embedding_dir)
    embeds = []
    for face_file in os.listdir(face_dir):
        face_path = os.path.join(face_dir, face_file)
        try:
            img = cv2.imread(face_path)
        except:
            continue
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = np.stack([img])
        embed = model.predict(img)
        embeds.append(embed) #1 anh, kich thuoc [1,512]
        if len(embeds) == 0:
            continue
    embedding = tf.concat(embeds, 0, name='concat')
    embedding = tf.reduce_mean(embedding, 0)
    embedding_path = os.path.join(embedding_dir, f'{folder}.pth')
    with open(embedding_path, 'wb') as file:
      # A new file will be created
      pickle.dump(embedding, file)
  
def tf_prepare_data_for_training():
  print('PROCESSING TRAIN preparing data step')
  #training dataset
  extract_and_save_faces(TRAIN_DATASET_DIR)
  tf_embedding_and_save(TRAIN_DATASET_DIR)
  print('DONE TRAIN preparing data step')

def tf_prepare_data_for_testing():
  print('PROCESSING TEST preparing data step')
  #testing dataset
  extract_and_save_faces(TEST_DATASET_DIR)
  tf_embedding_and_save(TEST_DATASET_DIR)
  print('DONE TEST preparing data step')

In [46]:
# Run preparing data
tf_prepare_data_for_training()
tf_prepare_data_for_testing()

PROCESSING TRAIN preparing data step
1/1 [==============================] - 0s 31ms/step
DONE TRAIN preparing data step
PROCESSING TEST preparing data step
1/1 [==============================] - 0s 23ms/step
DONE TEST preparing data step


# Training model (from scratch)

In [10]:
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Lambda, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16


In [ ]:
def _base_network():
  model = VGG16(include_top = True, weights = None)
  dense = Dense(128)(model.layers[-4].output)
  norm2 = Lambda(lambda x: tf.math.l2_normalize(x, axis = 1))(dense)
  model = Model(inputs = [model.input], outputs = [norm2])
  return model

model = _base_network()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
import tensorflow_addons as tfa

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
X_train, testing_faces, y_train, testing_labels = load_dataset_for_recognition()
gen_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().shuffle(1024).batch(32)
gen_train

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))>

In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch = 50,
    epochs=5)

In [ ]:
MODEL_DIR = os.path.join(BASE_DIR, 'model')
if os.path.exists(MODEL_DIR) != True:
    os.mkdir(MODEL_DIR)
model_name = 'face_recognition_triplot.h5'
model_path = os.path.join(MODEL_DIR, model_name)
model.save(model_path)

# Evaluation - Recognition



*   Đánh giá trong face recognition thì sẽ được tổng hợp từ 2 quá trình đánh giá:


1.   Đánh giá của bước face detection
2.   Đánh giá của bước face recognition







In [47]:
from sklearn.metrics import accuracy_score

# For pretrain (pytorch)

In [40]:
def load_embedding_dataset_for_deploy():
  result = {'embedding': [], 'user_name': []}
  for user in os.listdir(TRAIN_DATASET_DIR):
      EMBEDDING_FILE = os.path.join(TRAIN_DATASET_DIR, user, EMBEDDING_FOLDER, f'{user}.pth')
      embedding = torch.load(EMBEDDING_FILE)
      user_name = str(user)
      result['embedding'].append(embedding)
      result['user_name'].append(user_name)
  return result

def inference(face, model, embedding_dataset, threshold=0.8):
  local_embeds = torch.cat(embedding_dataset['embedding'])
  names = embedding_dataset['user_name']
  face = transfrom_img(face).to(device)
  face = face.unsqueeze(0)
  embed = model(face)
  norm_diff = embed - local_embeds
  norm_square = torch.pow(norm_diff, 2)
  norm_score = torch.sum(norm_square, dim=1) #(1,n)
  #norm_score = torch.sqrt(norm_score)
  embed_idx = torch.argmin(norm_score)
  min_dist = norm_score[embed_idx]
  if min_dist > threshold:
      return -1, -1
  else:
      return names[embed_idx], min_dist

def load_dataset_for_recognition():
  training_faces, testing_faces, training_labels, testing_labels = [], [], [], []
  for user in os.listdir(TRAIN_DATASET_DIR):
    faces_folder = os.path.join(TRAIN_DATASET_DIR, user, FACE_FOLDER)
    for face in os.listdir(faces_folder):
      face_path = os.path.join(faces_folder, face)
      face = cv2.imread(face_path)
      #for trainign model - not pretrained model
      #face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
      training_faces.append(face)
      training_labels.append(user)
  
  for user in os.listdir(TEST_DATASET_DIR):
    faces_folder = os.path.join(TEST_DATASET_DIR, user, FACE_FOLDER)
    for face in os.listdir(faces_folder):
      face_path = os.path.join(faces_folder, face)
      face = cv2.imread(face_path)
      #for trainign model - not pretrained model
      #face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
      testing_faces.append(face)
      testing_labels.append(user)
  return training_faces, testing_faces, training_labels, testing_labels

def recognition_evaluation(model):
  embedding_dataset = load_embedding_dataset_for_deploy()
  training_faces, testing_faces, training_labels, testing_labels = load_dataset_for_recognition()
  predicts = []
  for face in testing_faces:
    user, score = inference(face, model, embedding_dataset)
    if user == -1:
      user = 'unknown'
    else:
      # Using cv2.putText() method to display score
      score = torch.round(score, decimals=4)
    predicts.append(user)
  acc = accuracy_score(testing_labels, predicts)
  print('accuracy: ', acc)

In [73]:
#Evaluation execute
recognition_evaluation(model)

accuracy:  0.8846153846153846


# For my own model (tensorflow)

In [67]:
def tf_load_embedding_dataset_for_deploy():
  result = {'embedding': [], 'user_name': []}
  for user in os.listdir(TRAIN_DATASET_DIR):
      EMBEDDING_FILE = os.path.join(TRAIN_DATASET_DIR, user, EMBEDDING_FOLDER, f'{user}.pth')
      with open(EMBEDDING_FILE, 'rb') as file:
        # Call load method to deserialze
        embedding = pickle.load(file)
      user_name = str(user)
      result['embedding'].append(embedding)
      result['user_name'].append(user_name)
  return result

def tf_load_dataset_for_recognition():
  training_faces, testing_faces, training_labels, testing_labels = [], [], [], []
  for user in os.listdir(TRAIN_DATASET_DIR):
    faces_folder = os.path.join(TRAIN_DATASET_DIR, user, FACE_FOLDER)
    for face in os.listdir(faces_folder):
      face_path = os.path.join(faces_folder, face)
      face = cv2.imread(face_path)
      #for trainign model - not pretrained model
      face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
      training_faces.append(face)
      training_labels.append(user)
  
  for user in os.listdir(TEST_DATASET_DIR):
    faces_folder = os.path.join(TEST_DATASET_DIR, user, FACE_FOLDER)
    for face in os.listdir(faces_folder):
      face_path = os.path.join(faces_folder, face)
      face = cv2.imread(face_path)
      #for trainign model - not pretrained model
      face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
      testing_faces.append(face)
      testing_labels.append(user)
  return training_faces, testing_faces, training_labels, testing_labels

def tf_inference(face, model, embedding_dataset, threshold=0.8, verbose=1):
  local_embeds = embedding_dataset['embedding']
  names = embedding_dataset['user_name']
  #print(face.shape, local_embeds.shape)
  face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
  face = np.stack([face])
  embed = model.predict(face, verbose=verbose)
  norm_diff = embed - local_embeds
  norm_square = np.power(norm_diff, 2)
  norm_score = np.sum(norm_square, axis=1) #(1,n)
  #norm_score = torch.sqrt(norm_score)
  embed_idx = np.argmin(norm_score)
  min_dist = norm_score[embed_idx]
  if min_dist > threshold:
      return -1, -1
  else:
      return names[embed_idx], min_dist

def tf_recognition_evaluation(model):
  embedding_dataset = tf_load_embedding_dataset_for_deploy()
  training_faces, testing_faces, training_labels, testing_labels = tf_load_dataset_for_recognition()
  predicts = []
  for face in testing_faces:
    user, score = tf_inference(face, model, embedding_dataset)
    if user == -1:
      user = 'unknown'
    else:
      # Using cv2.putText() method to display score
      #score = torch.round(score, decimals=4)
      pass
    predicts.append(user)
  print(testing_labels)
  print(predicts)
  acc = accuracy_score(testing_labels, predicts)
  print('accuracy: ', acc)

In [51]:
#Evaluation execute
tf_recognition_evaluation(model)

1/1 [==============================] - 0s 23ms/step
['ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi']
['ronaldo', 'ronaldo', 'messi', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'ronaldo', 'messi', 'ronaldo', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi', 'messi']
accuracy:  0.9230769230769231


# Deploy

In [62]:
# Deploy with query method (compute simalarity between two embedding - input and my own dataset)
embedding_dataset = tf_load_embedding_dataset_for_deploy()

for user in os.listdir(TEST_DATASET_DIR):
  user_dir = os.path.join(TEST_DATASET_DIR, user, IMG_FOLDER)
  for img_file in os.listdir(user_dir):
    img_path = os.path.join(user_dir, img_file)
    img = cv2.imread(img_path)
    boxes, probs = mtcnn.detect(img)
    if boxes is not None:
      for box, prob in zip(boxes, probs):
        bbox = list(map(int,box.tolist()))
        face, status = extract_face(bbox, img, img.shape)
        if status == True:
          user, score = tf_inference(face, model, embedding_dataset)
          if user == -1:
            user = 'unknown'
            img = cv2.putText(img, f'{user}', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.3, (0, 0, 255), 1, cv2.LINE_AA)
            img = cv2.rectangle(img,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),1)
          else:
            print(img_file)
            # Using cv2.putText() method to display score
            #score = torch.round(score, decimals=4)
            img = cv2.putText(img, f'{user}: {score}', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.3, (0, 0, 255), 1, cv2.LINE_AA)
            img = cv2.rectangle(img,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),1)
        else:
          print('face detection is error')
    #image show in ipython
    plt.axis("off")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

# Deploy real-time on camera

In [63]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [64]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
#load_data_for_query
embedding_dataset = load_embedding_dataset_for_deploy()

# start streaming video from webcam
# infer with mtcnn on colab (using camera from latop)
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  
    # get face region coordinates
    bboxes, _ = mtcnn.detect(img)
    #bbox_array = img
    # get face bounding box for overlay
    if bboxes is not None:
      for box in bboxes:
          bbox = list(map(int,box.tolist()))
          
          face, status = extract_face(bbox, img, img.shape)
          if status == True:
            user, score = inference(face, model, embedding_dataset)
            #user, score = tf_inference(face, model, embedding_dataset, verbose=0)
            if user == -1:
              user = 'unknown'
              bbox_array = cv2.putText(bbox_array, f'{user}', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                          0.3, (0, 0, 255), 1, cv2.LINE_AA)
              bbox_array = cv2.rectangle(bbox_array,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),1)
            else:
              # Using cv2.putText() method to display score
              #score = torch.round(score, decimals=4)
              bbox_array = cv2.putText(bbox_array, f'{user}: {score}', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                          0.3, (0, 0, 255), 1, cv2.LINE_AA)
              bbox_array = cv2.rectangle(bbox_array,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),1)
            #bbox_array = cv2.rectangle(bbox_array,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),6)
          else:
            print('face detection is error')
    #for (x,y,w,h) in faces:
      #bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
    
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes